In [28]:
import nltk
print("NLTK version: {}".format(nltk.__version__))

text = """The Buddha, the Godhead, resides quite as comfortably in the circuits of a digital
computer or the gears of a cycle transmission as he does at the top of a mountain
or in the petals of a flower. To think otherwise is to demean the Buddha...which is
to demean oneself."""

# Used when tokenizing words
sentence_re = r'''[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\'\w\-]+'''

lemmatizer = nltk.WordNetLemmatizer()
stemmer = nltk.stem.porter.PorterStemmer()

#Taken from Su Nam Kim Paper...
grammar = r"""
    NBAR:
        {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
        
    NP:
        {<NBAR>}
        {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
"""
chunker = nltk.RegexpParser(grammar)

toks = nltk.regexp_tokenize(text, sentence_re)
postoks = nltk.tag.pos_tag(toks)

print(postoks)

NLTK version: 3.2.1
[('The', 'DT'), ('Buddha', 'NNP'), ('the', 'DT'), ('Godhead', 'NNP'), ('resides', 'VBZ'), ('quite', 'RB'), ('as', 'IN'), ('comfortably', 'RB'), ('in', 'IN'), ('the', 'DT'), ('circuits', 'NNS'), ('of', 'IN'), ('a', 'DT'), ('digital', 'JJ'), ('computer', 'NN'), ('or', 'CC'), ('the', 'DT'), ('gears', 'NNS'), ('of', 'IN'), ('a', 'DT'), ('cycle', 'NN'), ('transmission', 'NN'), ('as', 'IN'), ('he', 'PRP'), ('does', 'VBZ'), ('at', 'IN'), ('the', 'DT'), ('top', 'NN'), ('of', 'IN'), ('a', 'DT'), ('mountain', 'NN'), ('or', 'CC'), ('in', 'IN'), ('the', 'DT'), ('petals', 'NNS'), ('of', 'IN'), ('a', 'DT'), ('flower', 'NN'), ('To', 'TO'), ('think', 'VB'), ('otherwise', 'RB'), ('is', 'VBZ'), ('to', 'TO'), ('demean', 'VB'), ('the', 'DT'), ('Buddha', 'NNP'), ('which', 'WDT'), ('is', 'VBZ'), ('to', 'TO'), ('demean', 'VB'), ('oneself', 'PRP')]


In [44]:
tree = chunker.parse(postoks)

from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [48]:

def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label=='NP'):
        yield subtree.leaves()

def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    word = stemmer.stem_word(word)
    word = lemmatizer.lemmatize(word)
    return word

def acceptable_word(word):
    """Checks conditions for acceptable word: length, stopword."""
    accepted = bool(2 <= len(word) <= 40
        and word.lower() not in stopwords)
    return accepted


def get_terms(tree):
    for leaf in leaves(tree):
        term = [ normalise(w) for w,t in leaf if acceptable_word(w) ]
        yield term

terms = get_terms(tree)

for term in terms:
    for word in term:
        print(word),
    print